<a href="https://colab.research.google.com/github/vinodvicki/Weather_prediction-/blob/main/Rain_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# predecting the rain fall
# importing nessary libries
!pip install catboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix,classification_report,f1_score,precision_score,recall_score

from imblearn.over_sampling import SMOTE
from scipy.stats import stats



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 11.1 MB/s eta 0:00:00


In [3]:
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets /Machine Learning Project 32 - Rain Fall Prediction/weatherAUS.csv")

In [4]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

Your dataset contains weather-related features for rainfall prediction. It includes columns like:

* Date: The date of the observation.
* Location: The location of the weather observation.
* MinTemp and MaxTemp: Minimum and maximum temperatures of the day.
* Rainfall: Amount of rainfall in mm.
* WindGustDir, WindGustSpeed, WindDir9am, WindDir3pm, WindSpeed9am,   
* WindSpeed3pm: Wind-related metrics.
* Humidity9am and Humidity3pm: Humidity percentages at 9 AM and 3 PM.
* Pressure9am and Pressure3pm: Atmospheric pressure at 9 AM and 3 PM.
* Cloud9am and Cloud3pm: Cloud cover at 9 AM and 3 PM.
* Temp9am and Temp3pm: Temperatures at 9 AM and 3 PM.
* RainToday and RainTomorrow: Indicating if it rained today or if it will rain tomorrow.

In [6]:
#shuffel the data in random
df=df.sample(frac=1).reset_index(drop=True)

In [7]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2013-07-26,BadgerysCreek,0.3,19.7,0.0,NaN,NaN,N,13.0,NW,...,100.0,37.0,1025.8,1023.4,NaN,NaN,8.4,19.3,No,No
1,2009-11-04,Watsonia,9.5,18.4,1.4,6.8,3.7,W,39.0,SW,...,70.0,55.0,1019.4,1019.1,6.0,7.0,12.7,17.1,Yes,No
2,2015-01-27,Sale,12.5,18.9,0.0,NaN,NaN,S,26.0,S,...,64.0,56.0,1017.6,1016.9,NaN,8.0,16.0,18.6,No,No
3,2011-12-06,Ballarat,5.5,23.7,0.0,NaN,NaN,NNE,50.0,NNE,...,51.0,31.0,1017.4,1014.5,0.0,NaN,14.8,21.8,No,No
4,2012-10-08,MountGambier,6.0,15.3,0.2,1.2,6.2,SW,33.0,N,...,79.0,61.0,1018.4,1015.8,7.0,5.0,10.2,13.1,No,No


# Data preprocessing

In [9]:
# missing values
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_values / len(df)) * 100

missing_display = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage}).sort_values(by='Percentage', ascending=False )

print(missing_display)

               Missing Values  Percentage
Sunshine                69835   48.009762
Evaporation             62790   43.166506
Cloud3pm                59358   40.807095
Cloud9am                55888   38.421559
Pressure9am             15065   10.356799
Pressure3pm             15028   10.331363
WindDir9am              10566    7.263853
WindGustDir             10326    7.098859
WindGustSpeed           10263    7.055548
Humidity3pm              4507    3.098446
WindDir3pm               4228    2.906641
Temp3pm                  3609    2.481094
RainTomorrow             3267    2.245978
Rainfall                 3261    2.241853
RainToday                3261    2.241853
WindSpeed3pm             3062    2.105046
Humidity9am              2654    1.824557
WindSpeed9am             1767    1.214767
Temp9am                  1767    1.214767
MinTemp                  1485    1.020899
MaxTemp                  1261    0.866905
Date                        0    0.000000
Location                    0    0

* dropping columns with more than 40% missing data might lose valuable information. Instead, we can take alternative approaches to handle such columns while retaining their information

Alternative Strategies for Handling Columns with High Missing Values
Impute with Advanced Techniques

* Use statistical models or machine learning algorithms (e.g., KNN Imputation) to predict missing values based on other correlated features.
For numerical columns (e.g., Sunshine, Evaporation), impute using grouped statistics like the median or mean for similar weather conditions (e.g., grouped by Location or Month)

Treat Missing Data as a Separate Category

* For categorical features, encode missing values as a new category (Unknown).
For numerical columns, create a binary indicator column to flag missing values and impute the missing ones with the median or mean.

Use Domain Knowledge

* For features like Cloud3pm, where values may be correlated with other cloud-related features or time-based trends, estimate values using related features or models.

Use Predictive Imputation

* Train a regression or classification model to predict missing values using the available data

Cluster-Based Imputation

* Cluster data into similar groups (e.g., using K-means) and impute missing values within each cluster based on group-specific statistics.

Retain Columns but Limit Their Influence

* Standardize features with high missing data to reduce their variability and impact during modeling.

In [10]:
## Retaining columns with high missing values and handling them differently
from sklearn.impute import KNNImputer

# 1. Handle Missing Values:
# Identify numerical and categorical columns with missing values
numericial_columns = df.select_dtypes(include= np.number).columns
categorical_columns = df.select_dtypes(exclude = np.number).columns

In [11]:
# Using grouped median imputation for numerical columns based on 'Location' and 'Month
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
for col in numericial_columns:
    df[col] = df.groupby(['Location', 'Month'])[col].transform(lambda x: x.fillna(x.median()))# his part applies a function (lambda x: x.fillna(x.median()))
    #to each group. This function replaces missing values (NaN) in the selected column (col) with the median value of that column within the same group.

# impute missing numerical data with KNNimputer
imputer = KNNImputer(n_neighbors=5)
df[numericial_columns] = imputer.fit_transform(df[numericial_columns])

#impute missing Catagorical data with unknown
for column in categorical_columns:
    df[column].fillna('Unknown', inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  object        
 2   MinTemp        145460 non-null  float64       
 3   MaxTemp        145460 non-null  float64       
 4   Rainfall       145460 non-null  float64       
 5   Evaporation    145460 non-null  float64       
 6   Sunshine       145460 non-null  float64       
 7   WindGustDir    145460 non-null  object        
 8   WindGustSpeed  145460 non-null  float64       
 9   WindDir9am     145460 non-null  object        
 10  WindDir3pm     145460 non-null  object        
 11  WindSpeed9am   145460 non-null  float64       
 12  WindSpeed3pm   145460 non-null  float64       
 13  Humidity9am    145460 non-null  float64       
 14  Humidity3pm    145460 non-null  float64       
 15  

In [29]:
df[['WindGustDir','WindDir9am','WindDir3pm','Location']].apply(pd.unique)
# Get unique values for multiple columns using a loop:
for col in ['WindGustDir','WindDir9am','WindDir3pm','Location']:
    print(df[col].unique())

['N' 'W' 'S' 'NNE' 'SW' 'E' 'SSE' 'NE' 'WNW' 'ESE' 'Unknown' 'ENE' 'SE'
 'WSW' 'NW' 'NNW' 'SSW']
['NW' 'SW' 'S' 'NNE' 'N' 'ESE' 'ENE' 'W' 'NNW' 'NE' 'SSE' 'WSW' 'E' 'SSW'
 'Unknown' 'WNW' 'SE']
['N' 'S' 'SSE' 'W' 'SSW' 'E' 'NE' 'NNW' 'ESE' 'ENE' 'SE' 'NW' 'WSW' 'SW'
 'WNW' 'NNE' 'Unknown']
['BadgerysCreek' 'Watsonia' 'Sale' 'Ballarat' 'MountGambier' 'Dartmoor'
 'SalmonGums' 'Wollongong' 'Adelaide' 'AliceSprings' 'Cairns'
 'NorfolkIsland' 'Williamtown' 'Sydney' 'WaggaWagga' 'Perth' 'Newcastle'
 'Hobart' 'SydneyAirport' 'MountGinini' 'Penrith' 'Townsville' 'Uluru'
 'Melbourne' 'MelbourneAirport' 'PerthAirport' 'Walpole' 'Launceston'
 'Tuggeranong' 'Albury' 'CoffsHarbour' 'Moree' 'Albany' 'Darwin' 'Cobar'
 'GoldCoast' 'Katherine' 'Nuriootpa' 'Portland' 'Canberra' 'Mildura'
 'Brisbane' 'NorahHead' 'Bendigo' 'PearceRAAF' 'Richmond' 'Witchcliffe'
 'Woomera' 'Nhil']


In [ ]:
# ecncoding catagorical features for ordinal  and nominal accordingly.
df['RainToday'] = df['RainToday'].map({'No': 0, 'Yes': 1})
df['RainTomorrow'] = df['RainTomorrow'].map({'No': 0, 'Yes': 1})

#ecncoding catagorical features for nominal data
df = pd.get_dummies(df, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], drop_first=True)